#Librerie

In [ ]:

!pip install ucimlrepo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler



#Caricamento del Dataset

In [ ]:

from ucimlrepo import fetch_ucirepo

infrared_thermography_temperature = fetch_ucirepo(id=925)

X = infrared_thermography_temperature.data.features
y = infrared_thermography_temperature.data.targets


print(infrared_thermography_temperature.metadata)
print(infrared_thermography_temperature.variables)


##Prima visione dell'input

In [ ]:
X.head()
X.describe()

##Prima visione dell'output

In [ ]:
y.head()
y.describe()

##Train-test splitting

In [ ]:

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dimensione di X_train_raw: {X_train_raw.shape}")
print(f"Dimensione di y_train: {y_train.shape}")
print(f"Dimensione di X_test_raw: {X_test_raw.shape}")
print(f"Dimensione di y_test: {y_test.shape}")

In [ ]:
y_train.describe()

In [ ]:
y_test.describe()

#Preprocessing



In [ ]:

def preprocess_features(df_X, training_cols=None):
    df_X_copy = df_X.copy() # Lavora su una copia del DataFrame

    # Sostituisce i valori inf con NaN per primi
    df_X_copy.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Riempie i valori NaN per le colonne numeriche con la loro media (calcolata all'interno di ogni split per ora)
    for col in df_X_copy.select_dtypes(include=np.number).columns:
        if df_X_copy[col].isnull().any():
            df_X_copy[col] = df_X_copy[col].fillna(df_X_copy[col].mean())

    # One-Hot Encoding per le variabili categoriche
    categorical_cols = df_X_copy.select_dtypes(include='object').columns
    if not categorical_cols.empty:
        df_X_encoded = pd.get_dummies(df_X_copy, columns=categorical_cols, drop_first=True)
    else:
        df_X_encoded = df_X_copy

    return df_X_encoded

# Applica il preprocessing alle feature di training e test
X_train_encoded_df = preprocess_features(X_train_raw)
X_test_encoded_df = preprocess_features(X_test_raw)

# Inizializza StandardScaler
scaler = StandardScaler()

# Applica lo scaler a X_train_encoded_df e trasforma tutti i set
# Converte in array numpy dopo la scalatura
X_train = scaler.fit_transform(X_train_encoded_df)
X_test = scaler.transform(X_test_encoded_df)

# Funzione per il preprocessing dei dati y (target) - gestisce solo NaN/inf, nessuna normalizzazione
def preprocess_targets(df_y):
    df_y_copy = df_y.copy() # Lavora su una copia del DataFrame

    df_y_copy.replace([np.inf, -np.inf], np.nan, inplace=True)

    for col in df_y_copy.select_dtypes(include=np.number).columns:
        if df_y_copy[col].isnull().any():
            df_y_copy[col] = df_y_copy[col].fillna(df_y_copy[col].mean())

    return df_y_copy.values # Converte in array numpy

# Applica il preprocessing ai target di training e test
y_train = preprocess_targets(y_train)
y_test = preprocess_targets(y_test)

print(f"Dimensione di X_train (scalato): {X_train.shape}")
print(f"Dimensione di y_train (preprocessato): {y_train.shape}")
print(f"Dimensione di X_test (scalato): {X_test.shape}")
print(f"Dimensione di y_test (preprocessato): {y_test.shape}")



#EDA

##Matrice di correlazione e distribuzione target

In [ ]:

# Utilizziamo X_train_encoded_df che è il DataFrame con le colonne one-hot encoded
feature_names_for_eda = X_train_encoded_df.columns

# Recupera i nomi delle colonne delle variabili target originali
target_names = infrared_thermography_temperature.data.targets.columns

# Crea un DataFrame combinato per il training set per l'analisi di correlazione
# Usiamo X_train_encoded_df (non scalato) per un'EDA più significativa delle correlazioni
df_train_eda = pd.DataFrame(X_train_encoded_df, columns=feature_names_for_eda)
for i, name in enumerate(target_names):
    df_train_eda[name] = y_train[:, i]

print("### Matrice di Correlazione ###")
plt.figure(figsize=(20, 18))
sns.heatmap(df_train_eda.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matrice di Correlazione del Training Set (Pre-scalatura)')
plt.show()

print("\n### Distribuzione delle Label (Variabili Target) ###")
plt.figure(figsize=(14, 6))
for i, name in enumerate(target_names):
    plt.subplot(1, len(target_names), i + 1)
    sns.histplot(y_train[:, i], kde=True)
    plt.title(f'Distribuzione di {name}')
    plt.xlabel(name)
    plt.ylabel('Frequenza')
plt.tight_layout()
plt.show()


##Pairplot delle feature continue

In [ ]:

#Identifica le feature numeriche originali dal dataset non processato
native_numerical_feature_names = infrared_thermography_temperature.data.features.select_dtypes(include=np.number).columns

#Ottieni gli indici di queste feature all'interno di X_train_encoded_df
#Usiamo X_train_encoded_df per l'EDA perché è il DataFrame con feature one-hot encoded ma non scalate
processed_feature_names_for_eda = X_train_encoded_df.columns
indices_native_numerical = [processed_feature_names_for_eda.get_loc(col) for col in native_numerical_feature_names if col in processed_feature_names_for_eda]

#Crea un DataFrame per il pairplot usando solo queste feature da X_train_encoded_df
df_pairplot_numerical = pd.DataFrame(X_train_encoded_df.iloc[:, indices_native_numerical], columns=native_numerical_feature_names)

#Aggiungi le variabili target al DataFrame
target_names = infrared_thermography_temperature.data.targets.columns
for i, name in enumerate(target_names):
    df_pairplot_numerical[name] = y_train[:, i]

print("### Pairplot delle Feature Nativamente Numeriche e Target (Training Set) ###")
sns.pairplot(df_pairplot_numerical, diag_kind='kde')
plt.suptitle('Pairplot delle Feature Nativamente Numeriche e Target (Training Set)', y=1.02) # y modifica la posizione del titolo
plt.show()




#Training

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, r2_score

model_oralF = ElasticNet(l1_ratio=0.25, alpha=0.01, random_state=42)
model_oralM = ElasticNet(l1_ratio=0.25, alpha=0.01, random_state=42)

model_oralF.fit(X_train, y_train[:, 0])
model_oralM.fit(X_train, y_train[:, 1])

y_train_pred_oralF = model_oralF.predict(X_train)

#Calcola il Mean Absolute Error (MAE) e R^2 per aveOralF sul training set
mae_train_oralF = mean_absolute_error(y_train[:, 0], y_train_pred_oralF)
r2_train_oralF = r2_score(y_train[:, 0], y_train_pred_oralF)
print(f"MAE sul Training Set per aveOralF: {mae_train_oralF:.4f}")
print(f"R^2 sul Training Set per aveOralF: {r2_train_oralF:.4f}")

#Ripetere i passaggi per AveOralM
y_train_pred_oralM = model_oralM.predict(X_train)

mae_train_oralM = mean_absolute_error(y_train[:, 1], y_train_pred_oralM)
r2_train_oralM = r2_score(y_train[:, 1], y_train_pred_oralM)
print(f"MAE sul Training Set per aveOralM: {mae_train_oralM:.4f}")
print(f"R^2 sul Training Set per aveOralM: {r2_train_oralM:.4f}")

##Modello lineare senza regolarizzazione

In [ ]:
from sklearn.linear_model import LinearRegression

linear_model_oralF = LinearRegression()
linear_model_oralM = LinearRegression()

linear_model_oralF.fit(X_train, y_train[:, 0])
linear_model_oralM.fit(X_train, y_train[:, 1])

y_train_pred_oralF_linear = linear_model_oralF.predict(X_train)

mae_train_oralF_linear = mean_absolute_error(y_train[:, 0], y_train_pred_oralF_linear)
r2_train_oralF_linear = r2_score(y_train[:, 0], y_train_pred_oralF_linear)
print(f"Risultati del Modello Lineare (senza regolarizzazione) sul Training Set per aveOralF")
print(f"MAE per aveOralF: {mae_train_oralF_linear:.4f}")
print(f"R^2 per aveOralF: {r2_train_oralF_linear:.4f}")


y_train_pred_oralM_linear = linear_model_oralM.predict(X_train)

mae_train_oralM_linear = mean_absolute_error(y_train[:, 1], y_train_pred_oralM_linear)
r2_train_oralM_linear = r2_score(y_train[:, 1], y_train_pred_oralM_linear)
print(f"\nRisultati del Modello Lineare (senza regolarizzazione) sul Training Set per aveOralM")
print(f"MAE per aveOralM: {mae_train_oralM_linear:.4f}")
print(f"R^2 per aveOralM: {r2_train_oralM_linear:.4f}")


#Validazione

In [ ]:
from sklearn.model_selection import GridSearchCV

#Definire la griglia di parametri per GridSearchCV
param_grid = {
    'alpha': np.logspace(-4, 0, 5), # es., 0.0001, 0.001, 0.01, ecc..
    'l1_ratio': np.arange(0.0, 1.1, 0.25) # es., 0.0, 0.25, 0.5, ecc..
}

#Definire le metriche
metriche = {
    'neg_mae': 'neg_mean_absolute_error', # massimizzazione della metrica
    'r2': 'r2'
}

#Hyperparameter Tuning per aveOralF
print("Hyperparameter Tuning per aveOralF")
elastic_net_oralF = ElasticNet(random_state=42)
grid_search_oralF = GridSearchCV(
    elastic_net_oralF,
    param_grid,
    scoring=metriche, # Usare più metriche
    refit='neg_mae', # Riaddestra utilizzando MAE
    cv=5, # 5-fold cross-validation
    n_jobs=-1 # Usare tutti i core disponibili
)
grid_search_oralF.fit(X_train, y_train[:, 0])

print(f"Migliori iperparametri per aveOralF: {grid_search_oralF.best_params_}")
print(f"Miglior MAE per aveOralF (da CV): {-grid_search_oralF.best_score_:.4f}") # Convertire negative MAE
# Calcolare il punteggio R^2
best_oralF_idx = grid_search_oralF.cv_results_['rank_test_neg_mae'].argmin()
r2_oralF_best_params = grid_search_oralF.cv_results_['mean_test_r2'][best_oralF_idx]
print(f"R^2 per aveOralF (da CV con migliori iperparametri): {r2_oralF_best_params:.4f}")

#Hyperparameter Tuning per aveOralM
print("\nHyperparameter Tuning per aveOralM")
elastic_net_oralM = ElasticNet(random_state=42)
grid_search_oralM = GridSearchCV(
    elastic_net_oralM,
    param_grid,
    scoring=metriche,
    refit='neg_mae',
    cv=5,
    n_jobs=-1
)
grid_search_oralM.fit(X_train, y_train[:, 1])

print(f"Migliori iperparametri per aveOralM: {grid_search_oralM.best_params_}")
print(f"Miglior MAE per aveOralM (da CV): {-grid_search_oralM.best_score_:.4f}") # Convertire negative MAE
best_oralM_idx = grid_search_oralM.cv_results_['rank_test_neg_mae'].argmin()
r2_oralM_best_params = grid_search_oralM.cv_results_['mean_test_r2'][best_oralM_idx]
print(f"R^2 per aveOralM (da CV con migliori iperparametri): {r2_oralM_best_params:.4f}")

#Salviamo i migliori modelli trovati da GridSearchCV
best_model_oralF = grid_search_oralF.best_estimator_
best_model_oralM = grid_search_oralM.best_estimator_

#Testing

Predizioni di modello con regolarizzazione

In [ ]:
y_pred_oralF = best_model_oralF.predict(X_test)
y_pred_oralM = best_model_oralM.predict(X_test)

Predizioni di modello lineare senza regolarizzazione

In [ ]:
# Predizioni del modello lineare senza regolarizzazione sul test set
y_pred_oralF_linear = linear_model_oralF.predict(X_test)
y_pred_oralM_linear = linear_model_oralM.predict(X_test)


Testing e grafici

In [ ]:

#Valutazione per ElasticNet (aveOralF)
print("Valutazione ElasticNet per aveOralF")
mae_oralF_elastic = mean_absolute_error(y_test[:, 0], y_pred_oralF)
r2_oralF_elastic = r2_score(y_test[:, 0], y_pred_oralF)

print(f"MAE (aveOralF ElasticNet): {mae_oralF_elastic:.4f}")
print(f"R2 Score (aveOralF ElasticNet): {r2_oralF_elastic:.4f}")

# Analisi grafica dei Residui per aveOralF (ElasticNet)
residuals_oralF_elastic = y_test[:, 0] - y_pred_oralF
plt.figure(figsize=(10, 5))
sns.scatterplot(x=y_test[:, 0], y=residuals_oralF_elastic)
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Analisi dei Residui per aveOralF (ElasticNet)")
plt.xlabel("Valori Target Reali (aveOralF)")
plt.ylabel("Residui (aveOralF)")
plt.show()

#Valutazione per ElasticNet (aveOralM)
print("\nValutazione ElasticNet per aveOralM")
mae_oralM_elastic = mean_absolute_error(y_test[:, 1], y_pred_oralM)
r2_oralM_elastic = r2_score(y_test[:, 1], y_pred_oralM)

print(f"MAE (aveOralM ElasticNet): {mae_oralM_elastic:.4f}")
print(f"R2 Score (aveOralM ElasticNet): {r2_oralM_elastic:.4f}")

# Analisi Grafica dei Residui per aveOralM (ElasticNet)
residuals_oralM_elastic = y_test[:, 1] - y_pred_oralM
plt.figure(figsize=(10, 5))
sns.scatterplot(x=y_test[:, 1], y=residuals_oralM_elastic)
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Analisi dei Residui per aveOralM (ElasticNet)")
plt.xlabel("Valori Target(aveOralM)")
plt.ylabel("Residui (aveOralM)")
plt.show()

#Valutazione per Modello Lineare (aveOralF)
print("\nValutazione Modello Lineare (senza reg.) per aveOralF")
mae_oralF_linear = mean_absolute_error(y_test[:, 0], y_pred_oralF_linear)
r2_oralF_linear = r2_score(y_test[:, 0], y_pred_oralF_linear)

print(f"MAE (aveOralF Lineare): {mae_oralF_linear:.4f}")
print(f"R2 Score (aveOralF Lineare): {r2_oralF_linear:.4f}")

#Valutazione per Modello Lineare (aveOralM)
print("\nValutazione Modello Lineare (senza reg.) per aveOralM")
mae_oralM_linear = mean_absolute_error(y_test[:, 1], y_pred_oralM_linear)
r2_oralM_linear = r2_score(y_test[:, 1], y_pred_oralM_linear)

print(f"MAE (aveOralM Lineare): {mae_oralM_linear:.4f}")
print(f"R2 Score (aveOralM Lineare): {r2_oralM_linear:.4f}")


